In [1]:
from models.lag_ml_model import scaling_function,preprocess_data,classifier_models,train_model,grid_optimize_model,random_optimize_model,get_param_grids,get_data

In [5]:
from backtesting import Backtest,Strategy
def backtest(y_pred, X_test):
    def get_signal():
        return y_pred
    class MyStrategy(Strategy):
        def init(self):
            self.signal = self.I(get_signal)
        def next(self):
            if self.signal == 1:
                if not self.position:
                    self.buy()
            elif self.signal == 0:
                if self.position:
                    self.position.close()

    bt = Backtest(X_test,MyStrategy,cash=10000,commission=0.0002)
    stats = bt.run()
    return stats

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [6]:
df = get_data(path=r'C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_AAPL_from_2015-01-01_to_2025-03-01.csv')

In [7]:
df

,Date,Close,High,Low,Open,Volume,Direction,sentiment_score,shifted_direction
0,2015-01-02,27.332500,27.860001,26.837500,27.847500,212818400,-1,0.596757,-1.0
1,2015-01-05,26.562500,27.162500,26.352501,27.072500,257142000,-1,0.867890,-1.0
2,2015-01-06,26.565001,26.857500,26.157499,26.635000,263188400,-1,0.887742,-1.0
3,2015-01-07,26.937500,27.049999,26.674999,26.799999,160423600,-1,0.887742,-1.0
4,2015-01-08,27.972500,28.037500,27.174999,27.307501,237458000,-1,0.883123,-1.0
...,...,...,...,...,...,...,...,...,...
2549,2025-02-21,245.550003,248.690002,245.220001,245.949997,53197400,1,0.009650,1.0
2550,2025-02-24,247.100006,248.860001,244.419998,244.929993,51326400,1,0.700623,1.0
2551,2025-02-25,247.039993,250.000000,244.910004,248.000000,48013300,1,0.833926,1.0
2552,2025-02-26,240.360001,244.979996,239.130005,244.330002,44433600,1,0.478114,1.0


In [10]:
import pandas as pd

In [31]:
import os
dir = r'C:\Users\sachi\Documents\Researchcode\sentiment'

for path in os.listdir(dir):
    print(os.path.join(dir,path))

C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_AAPL_from_2015-01-01_to_2025-03-01.csv
C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_AMZN_from_2015-01-01_to_2025-03-01.csv
C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_GOOGL_from_2015-01-01_to_2025-03-01.csv
C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_MSFT_from_2015-01-01_to_2025-03-01.csv
C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_NFLX_from_2015-01-01_to_2025-03-01.csv
C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_NVDA_from_2015-01-01_to_2025-03-01.csv
C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_TSLA_from_2015-01-01_to_2025-03-01.csv


In [32]:
import os
dir = r'C:\Users\sachi\Documents\Researchcode\sentiment'

for paths in os.listdir(dir):
    path = os.path.join(dir,paths)
    ticker = path.split('_')[2]
    print(f"Processing {ticker}")
    df = get_data(path=path)
    X_train,X_test,y_train,y_test = preprocess_data(df)
    X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)
    print(X_train_scaled.shape,X_test_scaled.shape,y_train.shape,y_test.shape)
    classifier = classifier_models()
    results,y_pred_dict = train_model(classifiers=classifier, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
    print(results)
    best_base = results.loc[results['Accuracy'].idxmax(), 'Model']
    stats_l=[]
    for name,y_pred in y_pred_dict.items():
        print(f"Backtesting for {name}")
        stats = backtest(y_pred,X_test)
        print(stats)
        stats_l.append(stats)
    normal_df = pd.DataFrame(stats_l)
    normal_df.to_csv(f'C:/Users/sachi/Documents/Researchcode/sentiment/{ticker}_normal.csv')


   






Processing AAPL
(1745, 66) (749, 66) (1745,) (749,)

Logistic Regression Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest Performance:
Accuracy: 0.6128
Confusion Matrix:
[[283  72]
 [218 176]]
              precision    recall  f1-score   support

           0       0.56      0.80      0.66       355
           1       0.71      0.45      0.55       394

    accuracy                           0.61       749
   macro avg       0.64      0.62      0.60       749
weighted avg       0.64      0.61      0.60       749


XGBoost Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.6061
Confusion Matrix:
[[264  91]
 [204 190]]
              precision    recall  f1-score   support

           0       0.56      0.74      0.64       355
           1       0.68      0.48      0.56       394

    accuracy                           0.61       749
   macro avg       0.62      0.61      0.60       749
weighted avg       0.62      0.61      0.60       749


K-Nearest Neighbors Performance:
Accuracy: 0.6649
Confusion Matrix:
[[217 138]
 [113 281]]
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       355
           1       0.67      0.71      0.69       394

    accuracy                           0.66       749
   macro avg       0.66      0.66      0.66       749
weighted avg       0.66      0.66      0.66       749


Decision Tree Performance:
Accuracy: 0.5794
Confusion Matrix:
[[ 59 296]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.76

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



MLP Performance:
Accuracy: 0.8758
Confusion Matrix:
[[299  56]
 [ 37 357]]
              precision    recall  f1-score   support

           0       0.89      0.84      0.87       355
           1       0.86      0.91      0.88       394

    accuracy                           0.88       749
   macro avg       0.88      0.87      0.88       749
weighted avg       0.88      0.88      0.88       749

[LightGBM] [Info] Number of positive: 1098, number of negative: 647
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16830
[LightGBM] [Info] Number of data points in the train set: 1745, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.629226 -> initscore=0.528899
[LightGBM] [Info] Start training from score 0.528899
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

...

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    68.89186
Equity Final [$]                   17852.0264
Equity Peak [$]                      19858.82
Commissions [$]                      91.62539
Return [%]                           78.52026
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                      21.529
Volatility (Ann.) [%]                61.50811
CAGR [%]                             14.32336
Sharpe Ratio                          0.35002
Sortino Ratio                         0.68955
Calmar Ratio                          0.41243
Alpha [%]                             77.9587
Beta                                  0.62269
Max. Drawdown [%]                   -52.19991
Avg. Drawdown [%]                   -13.58741
Max. Drawdown Duration      486 days 00:00:00
Avg. Drawdown Duration       78 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    68.89186
Equity Final [$]                   17852.0264
Equity Peak [$]                      19858.82
Commissions [$]                      91.62539
Return [%]                           78.52026
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                      21.529
Volatility (Ann.) [%]                61.50811
CAGR [%]                             14.32336
Sharpe Ratio                          0.35002
Sortino Ratio                         0.68955
Calmar Ratio                          0.41243
Alpha [%]                             77.9587
Beta                                  0.62269
Max. Drawdown [%]                   -52.19991
Avg. Drawdown [%]                   -13.58741
Max. Drawdown Duration      486 days 00:00:00
Avg. Drawdown Duration       78 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    39.65287
Equity Final [$]                   27098.3637
Equity Peak [$]                   33297.18565
Commissions [$]                     118.39715
Return [%]                          170.98364
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                     39.8502
Volatility (Ann.) [%]                51.32426
CAGR [%]                             25.89298
Sharpe Ratio                          0.77644
Sortino Ratio                         1.78981
Calmar Ratio                           1.1419
Alpha [%]                           170.66785
Beta                                  0.35016
Max. Drawdown [%]                   -34.89813
Avg. Drawdown [%]                   -10.19187
Max. Drawdown Duration      506 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    46.72897
Equity Final [$]                  16495.35661
Equity Peak [$]                   21253.68451
Commissions [$]                      95.38522
Return [%]                           64.95357
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                    18.33986
Volatility (Ann.) [%]                46.53866
CAGR [%]                             12.25517
Sharpe Ratio                          0.39408
Sortino Ratio                         0.74282
Calmar Ratio                          0.45758
Alpha [%]                            64.59825
Beta                                  0.39399
Max. Drawdown [%]                   -40.08053
Avg. Drawdown [%]                   -13.50134
Max. Drawdown Duration      534 days 00:00:00
Avg. Drawdown Duration      117 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    57.27637
Equity Final [$]                  19700.66152
Equity Peak [$]                    29176.5106
Commissions [$]                     171.17864
Return [%]                           97.00662
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                    25.62546
Volatility (Ann.) [%]                55.08379
CAGR [%]                             16.95516
Sharpe Ratio                          0.46521
Sortino Ratio                         0.96412
Calmar Ratio                           0.6211
Alpha [%]                            96.58175
Beta                                  0.47111
Max. Drawdown [%]                   -41.25833
Avg. Drawdown [%]                   -11.27686
Max. Drawdown Duration      478 days 00:00:00
Avg. Drawdown Duration       64 days 00:00:00
# Trades                          

In [ ]:
df = get_data(path=r'C:\Users\sachi\Documents\Researchcode\sentiment\merged_data_AAPL_from_2015-01-01_to_2025-03-01.csv')
X_train,X_test,y_train,y_test = preprocess_data(df)
X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)
print(X_train_scaled.shape,X_test_scaled.shape,y_train.shape,y_test.shape)
classifier = classifier_models()
results,y_pred_dict = train_model(classifiers=classifier, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
best_base = results.loc[results['Accuracy'].idxmax(), 'Model']
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)
    normal_df = pd.DataFrame(stats_l)








(1745, 66) (749, 66) (1745,) (749,)

Logistic Regression Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest Performance:
Accuracy: 0.6248
Confusion Matrix:
[[255 100]
 [181 213]]
              precision    recall  f1-score   support

           0       0.58      0.72      0.64       355
           1       0.68      0.54      0.60       394

    accuracy                           0.62       749
   macro avg       0.63      0.63      0.62       749
weighted avg       0.64      0.62      0.62       749


XGBoost Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f1-score   suppor

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.5808
Confusion Matrix:
[[249 106]
 [208 186]]
              precision    recall  f1-score   support

           0       0.54      0.70      0.61       355
           1       0.64      0.47      0.54       394

    accuracy                           0.58       749
   macro avg       0.59      0.59      0.58       749
weighted avg       0.59      0.58      0.58       749


K-Nearest Neighbors Performance:
Accuracy: 0.6649
Confusion Matrix:
[[217 138]
 [113 281]]
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       355
           1       0.67      0.71      0.69       394

    accuracy                           0.66       749
   macro avg       0.66      0.66      0.66       749
weighted avg       0.66      0.66      0.66       749


Decision Tree Performance:
Accuracy: 0.5794
Confusion Matrix:
[[ 59 296]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.76

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



MLP Performance:
Accuracy: 0.8611
Confusion Matrix:
[[288  67]
 [ 37 357]]
              precision    recall  f1-score   support

           0       0.89      0.81      0.85       355
           1       0.84      0.91      0.87       394

    accuracy                           0.86       749
   macro avg       0.86      0.86      0.86       749
weighted avg       0.86      0.86      0.86       749

[LightGBM] [Info] Number of positive: 1098, number of negative: 647
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001603 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16830
[LightGBM] [Info] Number of data points in the train set: 1745, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.629226 -> initscore=0.528899
[LightGBM] [Info] Start training from score 0.528899
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

...

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                     17.6235
Equity Final [$]                  11233.85153
Equity Peak [$]                   11233.85153
Commissions [$]                      69.90686
Return [%]                           12.33852
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     3.99209
Volatility (Ann.) [%]                 9.84117
CAGR [%]                              2.72382
Sharpe Ratio                          0.40565
Sortino Ratio                         0.66699
Calmar Ratio                          0.33989
Alpha [%]                             6.92767
Beta                                   0.1191
Max. Drawdown [%]                   -11.74533
Avg. Drawdown [%]                    -2.74711
Max. Drawdown Duration      274 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                  14863.79037
Equity Peak [$]                    16210.4295
Return [%]                            48.6379
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    14.26488
Volatility (Ann.) [%]                 30.9169
CAGR [%]                              9.58689
Sharpe Ratio                          0.46139
Sortino Ratio                         0.77786
Calmar Ratio                          0.47972
Alpha [%]                             3.99676
Beta                                  0.98261
Max. Drawdown [%]                   -29.73563
Avg. Drawdown [%]                    -5.70212
Max. Drawdown Duration      429 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                    0
Win Rate [%]                      

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    48.99866
Equity Final [$]                  12199.07299
Equity Peak [$]                   13671.99767
Commissions [$]                      73.31629
Return [%]                           21.99073
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     6.91645
Volatility (Ann.) [%]                16.63864
CAGR [%]                              4.69835
Sharpe Ratio                          0.41569
Sortino Ratio                         0.65798
Calmar Ratio                          0.34842
Alpha [%]                             6.96923
Beta                                  0.33064
Max. Drawdown [%]                   -19.85113
Avg. Drawdown [%]                    -4.95064
Max. Drawdown Duration      317 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    62.61682
Equity Final [$]                  10316.33518
Equity Peak [$]                   12124.39832
Commissions [$]                      59.31469
Return [%]                            3.16335
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     1.05333
Volatility (Ann.) [%]                17.94071
CAGR [%]                              0.72195
Sharpe Ratio                          0.05871
Sortino Ratio                         0.08406
Calmar Ratio                          0.04223
Alpha [%]                           -15.44451
Beta                                  0.40958
Max. Drawdown [%]                   -24.94095
Avg. Drawdown [%]                    -4.88518
Max. Drawdown Duration      577 days 00:00:00
Avg. Drawdown Duration       82 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    61.68224
Equity Final [$]                  14097.94656
Equity Peak [$]                   15379.42573
Commissions [$]                      77.40136
Return [%]                           40.97947
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.24921
Volatility (Ann.) [%]                26.03006
CAGR [%]                              8.25604
Sharpe Ratio                          0.47058
Sortino Ratio                         0.77892
Calmar Ratio                          0.38187
Alpha [%]                             7.47389
Beta                                   0.7375
Max. Drawdown [%]                   -32.07703
Avg. Drawdown [%]                     -6.9128
Max. Drawdown Duration      664 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

In [34]:
param_grids = get_param_grids()
print("Working on optimizing with GridSearchCV")
best_models = grid_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
results,y_pred_dict = train_model(classifiers=best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)
grid_df = pd.DataFrame(stats_l)
param_grids = get_param_grids()
print("Working on optimizing with RandomSearchCV")
random_best_models = random_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
results,y_pred_dict = train_model(classifiers=random_best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats['Algorithm'] = name
    stats_l.append(stats)
random_df = pd.DataFrame(stats_l)

Working on optimizing with GridSearchCV
Optimizing Logistic Regression...
Best parameters for Logistic Regression: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Optimizing Random Forest...
Best parameters for Random Forest: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 50}
Optimizing XGBoost...
Optimizing SVM...
Best parameters for SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Optimizing Naive Bayes...
Best parameters for Naive Bayes: {'alpha': 0.1}
Optimizing Gradient Boosting...
Best parameters for Gradient Boosting: {'learning_rate': 0.05, 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Optimizing K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'algorithm': 'auto', 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Optimizing Decision Tree...
Best parameters for Decision Tree: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_split': 2}
Optimizing AdaBoost...
Best parameters fo

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\linear_model\_stochas

Best parameters for SGD Classifier: {'alpha': 0.001, 'learning_rate': 'optimal', 'loss': 'modified_huber', 'penalty': 'l1'}
Optimizing MLP...
Best parameters for MLP: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.01, 'solver': 'sgd'}
Optimizing LGBMClassifier...
[LightGBM] [Info] Number of positive: 940, number of negative: 805
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001580 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16644
[LightGBM] [Info] Number of data points in the train set: 1745, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.538682 -> initscore=0.155038
[LightGBM] [Info] Start training from score 0.155038
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.7130
Confusion Matrix:
[[305 119]
 [ 96 229]]
              precision    recall  f1-score   support

           0       0.76      0.72      0.74       424
           1       0.66      0.70      0.68       325

    accuracy                           0.71       749
   macro avg       0.71      0.71      0.71       749
weighted avg       0.72      0.71      0.71       749


K-Nearest Neighbors Performance:
Accuracy: 0.6101
Confusion Matrix:
[[201 223]
 [ 69 256]]
              precision    recall  f1-score   support

           0       0.74      0.47      0.58       424
           1       0.53      0.79      0.64       325

    accuracy                           0.61       749
   macro avg       0.64      0.63      0.61       749
weighted avg       0.65      0.61      0.60       749


Decision Tree Performance:
Accuracy: 0.6115
Confusion Matrix:
[[284 140]
 [151 174]]
              precision    recall  f1-score   support

           0       0.65

...

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    58.47797
Equity Final [$]                  18322.38162
Equity Peak [$]                   22123.95361
Commissions [$]                     210.35558
Return [%]                           83.22382
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                    22.59702
Volatility (Ann.) [%]                 52.8504
CAGR [%]                             15.01216
Sharpe Ratio                          0.42757
Sortino Ratio                         0.85716
Calmar Ratio                          0.56508
Alpha [%]                            82.80231
Beta                                  0.46739
Max. Drawdown [%]                     -39.989
Avg. Drawdown [%]                   -12.58005
Max. Drawdown Duration      357 days 00:00:00
Avg. Drawdown Duration       70 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    64.08545
Equity Final [$]                   20269.8479
Equity Peak [$]                   22570.31861
Commissions [$]                     102.64023
Return [%]                          102.69848
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                    26.83509
Volatility (Ann.) [%]                61.72191
CAGR [%]                             17.72713
Sharpe Ratio                          0.43477
Sortino Ratio                         0.90208
Calmar Ratio                          0.53543
Alpha [%]                           102.17434
Beta                                  0.58118
Max. Drawdown [%]                   -50.11883
Avg. Drawdown [%]                   -14.21509
Max. Drawdown Duration      489 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    53.00401
Equity Final [$]                  14826.62166
Equity Peak [$]                   19683.70502
Commissions [$]                     122.69219
Return [%]                           48.26622
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                    14.16866
Volatility (Ann.) [%]                 48.1401
CAGR [%]                              9.52354
Sharpe Ratio                          0.29432
Sortino Ratio                         0.53165
Calmar Ratio                          0.29886
Alpha [%]                            47.86817
Beta                                  0.44137
Max. Drawdown [%]                   -47.40974
Avg. Drawdown [%]                   -14.72951
Max. Drawdown Duration      514 days 00:00:00
Avg. Drawdown Duration       82 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    47.79706
Equity Final [$]                  24874.56175
Equity Peak [$]                   29004.92682
Commissions [$]                      128.5763
Return [%]                          148.74562
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                    35.87869
Volatility (Ann.) [%]                53.62292
CAGR [%]                             23.42749
Sharpe Ratio                          0.66909
Sortino Ratio                         1.46687
Calmar Ratio                          0.90602
Alpha [%]                            148.3891
Beta                                  0.39533
Max. Drawdown [%]                   -39.60039
Avg. Drawdown [%]                   -10.02389
Max. Drawdown Duration      510 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    57.27637
Equity Final [$]                  19700.66152
Equity Peak [$]                    29176.5106
Commissions [$]                     171.17864
Return [%]                           97.00662
Buy & Hold Return [%]                 0.90184
Return (Ann.) [%]                    25.62546
Volatility (Ann.) [%]                55.08379
CAGR [%]                             16.95516
Sharpe Ratio                          0.46521
Sortino Ratio                         0.96412
Calmar Ratio                           0.6211
Alpha [%]                            96.58175
Beta                                  0.47111
Max. Drawdown [%]                   -41.25833
Avg. Drawdown [%]                   -11.27686
Max. Drawdown Duration      478 days 00:00:00
Avg. Drawdown Duration       64 days 00:00:00
# Trades                          

In [43]:
random_df

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades,Algorithm
0,2022-03-04,2025-02-27,1091 days,51.935915,23357.303175,28889.052422,208.490542,133.573032,0.901843,33.031749,...,66 days,17 days,2.343498,3.775764,0.991390,0.131877,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Logistic Regression
1,2022-03-04,2025-02-27,1091 days,38.451268,14368.409623,20495.440080,78.661163,43.684096,0.901843,12.969170,...,64 days,31 days,1.883216,4.654879,0.403814,0.141006,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Random Forest
2,2022-03-04,2025-02-27,1091 days,44.459279,18511.908414,27187.596633,153.887675,85.119084,0.901843,23.022225,...,73 days,21 days,2.048017,4.487624,0.612695,0.156761,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,XGBoost
3,2022-03-04,2025-02-27,1091 days,50.066756,18537.213188,25011.937762,132.454552,85.372132,0.901843,23.078779,...,73 days,24 days,2.108863,4.216247,0.765714,0.124898,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,SVM
4,2022-03-04,2025-02-27,1091 days,0.000000,10618.651493,17941.320499,NaN,6.186515,0.901843,2.040130,...,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit...",Naive Bayes
5,2022-03-04,2025-02-27,1091 days,45.260347,17557.484178,23375.465620,207.077025,75.574842,0.901843,20.850651,...,77 days,15 days,1.917320,2.853215,0.616688,0.107857,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Gradient Boosting
6,2022-03-04,2025-02-27,1091 days,62.349800,14203.821302,21149.227546,60.762218,42.038213,0.901843,12.532124,...,204 days,55 days,2.547055,11.451632,0.781601,0.198651,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,K-Nearest Neighbors
7,2022-03-04,2025-02-27,1091 days,58.477971,18322.381618,22123.953609,210.355580,83.223816,0.901843,22.597016,...,109 days,15 days,1.799072,2.356215,0.837171,0.131448,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,Decision Tree
8,2022-03-04,2025-02-27,1091 days,64.085447,20269.847898,22570.318609,102.640231,102.698479,0.901843,26.835089,...,248 days,38 days,2.933969,5.046116,1.155081,0.257421,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,AdaBoost
9,2022-03-04,2025-02-27,1091 days,53.004005,14826.621664,19683.705024,122.692194,48.266217,0.901843,14.168662,...,70 days,21 days,1.635558,2.650060,0.540631,0.070519,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...,SGD Classifier


In [52]:
df

,Date,Close,High,Low,Open,Volume,Direction,sentiment_score,shifted_direction
0,2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000,-1,0.887742,-1.0
1,2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500,-1,0.887742,-1.0
2,2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500,-1,0.887742,-1.0
3,2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000,-1,0.887742,-1.0
4,2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500,-1,0.887742,-1.0
...,...,...,...,...,...,...,...,...,...
2549,2025-02-21,337.799988,354.980011,334.420013,353.440002,74058600,-1,0.887742,-1.0
2550,2025-02-24,330.529999,342.399994,324.700012,338.140015,76052300,-1,0.887742,-1.0
2551,2025-02-25,302.799988,328.890015,297.250000,327.019989,134228800,-1,0.887742,-1.0
2552,2025-02-26,290.799988,309.000000,288.040009,303.709991,100118300,-1,0.887742,-1.0


In [53]:
df[df['Date'] == '2022-03-11']

,Date,Close,High,Low,Open,Volume,Direction,sentiment_score,shifted_direction
1810,2022-03-11,265.116669,281.266663,264.589996,280.066681,67037100,-1,0.918881,-1.0


In [49]:
X_test[X_test.index == '2022-03-11']

,Close,High,Low,Open,Volume,sentiment_score,Close_lag1,Close_lag2,Close_lag3,Close_lag4,...,Close_lag51,Close_lag52,Close_lag53,Close_lag54,Close_lag55,Close_lag56,Close_lag57,Close_lag58,Close_lag59,Close_lag60
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-11,265.116669,281.266663,264.589996,280.066681,67037100,0.918881,279.433319,286.323334,274.799988,268.193329,...,362.823334,364.646667,355.666656,336.290009,312.843323,299.980011,310.856659,308.973328,325.329987,319.503326


In [42]:
random_df._trades[0]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,Entry_get_signal,Exit_get_signal
0,35,5,6,280.066681,260.203339,None,None,-695.216980,-0.070924,2022-03-11,2022-03-14,3 days,None,0,0
1,30,11,28,304.993347,327.026672,None,None,660.999756,0.072242,2022-03-21,2022-04-13,23 days,None,1,0
2,28,32,33,343.333344,358.243347,None,None,417.480103,0.043427,2022-04-20,2022-04-21,1 days,None,0,0
3,43,77,80,237.470001,230.500000,None,None,-299.710052,-0.029351,2022-06-24,2022-06-29,5 days,None,1,0
4,45,83,122,223.000000,282.829987,None,None,2692.349396,0.268296,2022-07-05,2022-08-29,55 days,None,1,1
5,44,123,124,287.869995,280.619995,None,None,-319.000000,-0.025185,2022-08-30,2022-08-31,1 days,None,0,0
6,41,131,134,300.720001,301.829987,None,None,45.509399,0.003691,2022-09-12,2022-09-15,3 days,None,1,1
7,41,135,140,299.609985,283.089996,None,None,-677.319550,-0.055138,2022-09-16,2022-09-23,7 days,None,1,0
8,64,187,188,182.429993,197.080002,None,None,937.600586,0.080305,2022-11-30,2022-12-01,1 days,None,0,1
9,66,189,194,191.779999,173.839996,None,None,-1184.040161,-0.093545,2022-12-02,2022-12-09,7 days,None,1,0


In [38]:
normal_df.columns

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Commissions [$]', 'Return [%]',
       'Buy & Hold Return [%]', 'Return (Ann.) [%]', 'Volatility (Ann.) [%]',
       'CAGR [%]', 'Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio',
       'Alpha [%]', 'Beta', 'Max. Drawdown [%]', 'Avg. Drawdown [%]',
       'Max. Drawdown Duration', 'Avg. Drawdown Duration', '# Trades',
       'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]', 'Avg. Trade [%]',
       'Max. Trade Duration', 'Avg. Trade Duration', 'Profit Factor',
       'Expectancy [%]', 'SQN', 'Kelly Criterion', '_strategy',
       '_equity_curve', '_trades'],
      dtype='object')

In [41]:
normal_df._trades[0]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,SL,TP,PnL,ReturnPct,EntryTime,ExitTime,Duration,Tag,Entry_get_signal,Exit_get_signal
0,32,11,29,304.993347,333.096680,None,None,899.306641,0.092144,2022-03-21,2022-04-14,24 days,None,1,0
1,31,32,33,343.333344,358.243347,None,None,462.210114,0.043427,2022-04-20,2022-04-21,1 days,None,0,0
2,48,68,69,235.156662,223.166672,None,None,-575.519531,-0.050987,2022-06-10,2022-06-13,3 days,None,0,0
3,45,77,123,237.470001,287.869995,None,None,2267.999725,0.212237,2022-06-24,2022-08-30,67 days,None,1,0
4,44,132,133,292.899994,292.239990,None,None,-29.040161,-0.002253,2022-09-13,2022-09-14,1 days,None,0,0
5,43,135,141,299.609985,271.829987,None,None,-1194.539948,-0.092721,2022-09-16,2022-09-26,10 days,None,1,0
6,61,189,194,191.779999,173.839996,None,None,-1094.340149,-0.093545,2022-12-02,2022-12-09,7 days,None,1,0
7,83,221,253,128.679993,185.039993,None,None,4677.880051,0.437986,2023-01-20,2023-03-08,47 days,None,1,0
8,76,271,272,199.910004,197.320007,None,None,-196.839722,-0.012956,2023-04-03,2023-04-04,1 days,None,0,0
9,87,297,301,172.550003,165.649994,None,None,-600.300797,-0.039988,2023-05-10,2023-05-16,6 days,None,1,0


In [13]:
grid_df

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2022-03-04,2025-02-27,1091 days,54.873164,9565.934383,12249.298394,94.914136,-4.340656,45.431149,-1.481961,...,-0.084792,145 days,24 days,1.068750,0.176051,-0.049592,-0.007275,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
1,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
2,2022-03-04,2025-02-27,1091 days,27.636849,14093.888193,15375.367365,9.941782,40.938882,45.431149,12.238334,...,-7.266414,190 days,98 days,0.009044,-6.907929,-1.182126,-44.721951,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...
3,2022-03-04,2025-02-27,1091 days,52.336449,11979.267743,13701.264070,90.707834,19.792677,45.431149,6.264390,...,0.992927,144 days,26 days,1.565348,1.273079,0.613458,0.102120,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
4,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
5,2022-03-04,2025-02-27,1091 days,42.456609,13709.087108,13750.692977,117.543110,37.090871,45.431149,11.197835,...,1.222859,76 days,16 days,2.022184,1.410994,1.097030,0.233347,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
6,2022-03-04,2025-02-27,1091 days,59.946595,9162.904819,11450.509530,210.584714,-8.370952,45.431149,-2.898461,...,-0.120530,94 days,11 days,0.971335,-0.036693,-0.221261,-0.045724,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
7,2022-03-04,2025-02-27,1091 days,93.858478,14486.770209,15789.969366,47.554994,44.867702,45.431149,13.281414,...,4.188021,454 days,92 days,2.760199,4.714508,1.525510,0.418099,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
8,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
9,2022-03-04,2025-02-27,1091 days,51.134846,12045.922264,13284.775569,90.485900,20.459223,45.431149,6.462957,...,1.009491,143 days,26 days,1.609884,1.260375,0.675641,0.127857,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...


In [14]:
random_df

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2022-03-04,2025-02-27,1091 days,54.873164,9565.934383,12249.298394,94.914136,-4.340656,45.431149,-1.481961,...,-0.084792,145 days,24 days,1.068750,0.176051,-0.049592,-0.007275,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
1,2022-03-04,2025-02-27,1091 days,76.902537,9539.366783,11310.338576,101.539935,-4.606332,45.431149,-1.574103,...,0.138870,208 days,29 days,1.209561,0.461228,0.099526,0.019738,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
2,2022-03-04,2025-02-27,1091 days,27.636849,14093.888193,15375.367365,9.941782,40.938882,45.431149,12.238334,...,-7.266414,190 days,98 days,0.009044,-6.907929,-1.182126,-44.721951,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...
3,2022-03-04,2025-02-27,1091 days,52.336449,11979.267743,13701.264070,90.707834,19.792677,45.431149,6.264390,...,0.992927,144 days,26 days,1.565348,1.273079,0.613458,0.102120,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
4,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
5,2022-03-04,2025-02-27,1091 days,42.990654,13545.929375,14615.694320,126.352120,35.459294,45.431149,10.750804,...,1.183544,175 days,16 days,2.206560,1.493114,0.838888,0.238481,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
6,2022-03-04,2025-02-27,1091 days,59.946595,9162.904819,11450.509530,210.584714,-8.370952,45.431149,-2.898461,...,-0.120530,94 days,11 days,0.971335,-0.036693,-0.221261,-0.045724,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
7,2022-03-04,2025-02-27,1091 days,17.623498,11233.851528,11233.851528,69.906864,12.338515,45.431149,3.992089,...,0.690809,42 days,9 days,2.062746,0.726673,1.107638,0.348700,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
8,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
9,2022-03-04,2025-02-27,1091 days,48.998665,12199.072994,13671.997674,73.316292,21.990730,45.431149,6.916453,...,1.325121,143 days,31 days,1.709335,1.617260,0.727706,0.132136,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...


In [7]:
X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)
print(X_train_scaled.shape,X_test_scaled.shape,y_train.shape,y_test.shape)
classifier = classifier_models()
results,y_pred_dict = train_model(classifiers=classifier, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
best_base = results.loc[results['Accuracy'].idxmax(), 'Model']


(1745, 66) (749, 66) (1745,) (749,)

Logistic Regression Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest Performance:
Accuracy: 0.6328
Confusion Matrix:
[[262  93]
 [182 212]]
              precision    recall  f1-score   support

           0       0.59      0.74      0.66       355
           1       0.70      0.54      0.61       394

    accuracy                           0.63       749
   macro avg       0.64      0.64      0.63       749
weighted avg       0.65      0.63      0.63       749


XGBoost Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f1-score   suppor

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.5794
Confusion Matrix:
[[250 105]
 [210 184]]
              precision    recall  f1-score   support

           0       0.54      0.70      0.61       355
           1       0.64      0.47      0.54       394

    accuracy                           0.58       749
   macro avg       0.59      0.59      0.58       749
weighted avg       0.59      0.58      0.57       749


K-Nearest Neighbors Performance:
Accuracy: 0.6649
Confusion Matrix:
[[217 138]
 [113 281]]
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       355
           1       0.67      0.71      0.69       394

    accuracy                           0.66       749
   macro avg       0.66      0.66      0.66       749
weighted avg       0.66      0.66      0.66       749


Decision Tree Performance:
Accuracy: 0.5794
Confusion Matrix:
[[ 59 296]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.76

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [8]:
y_pred_dict

{'Logistic Regression': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1

In [9]:
from backtesting import Backtest,Strategy
def backtest(y_pred, X_test):
    def get_signal():
        return y_pred
    class MyStrategy(Strategy):
        def init(self):
            self.signal = self.I(get_signal)
        def next(self):
            if self.signal == 1:
                if not self.position:
                    self.buy()
            elif self.signal == 0:
                if self.position:
                    self.position.close()

    bt = Backtest(X_test,MyStrategy,cash=10000,commission=0.0002)
    stats = bt.run()
    return stats

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [10]:
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)


import pandas as pd
pd.DataFrame(stats_l)

Backtesting for Logistic Regression


Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    55.00668
Equity Final [$]                     9894.181
Equity Peak [$]                   12249.29839
Commissions [$]                      87.26928
Return [%]                           -1.05819
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    -0.35728
Volatility (Ann.) [%]                17.30052
CAGR [%]                             -0.24542
Sharpe Ratio                         -0.02065
Sortino Ratio                         -0.0284
Calmar Ratio                         -0.01659
Alpha [%]                           -18.72363
Beta                                  0.38884
Max. Drawdown [%]                   -21.53146
Avg. Drawdown [%]                     -6.7302
Max. Drawdown Duration      344 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    44.19226
Equity Final [$]                  12676.14343
Equity Peak [$]                   12676.14343
Commissions [$]                     105.00642
Return [%]                           26.76143
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     8.30534
Volatility (Ann.) [%]                18.83336
CAGR [%]                              5.63019
Sharpe Ratio                          0.44099
Sortino Ratio                         0.74206
Calmar Ratio                           0.2701
Alpha [%]                             8.24957
Beta                                  0.40747
Max. Drawdown [%]                   -30.74883
Avg. Drawdown [%]                     -5.5736
Max. Drawdown Duration      317 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    27.63685
Equity Final [$]                  14093.88819
Equity Peak [$]                   15375.36737
Commissions [$]                       9.94178
Return [%]                           40.93888
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.23833
Volatility (Ann.) [%]                30.08641
CAGR [%]                              8.24884
Sharpe Ratio                          0.40677
Sortino Ratio                         0.67242
Calmar Ratio                           0.4172
Alpha [%]                            -2.71274
Beta                                  0.96083
Max. Drawdown [%]                   -29.33467
Avg. Drawdown [%]                    -6.80479
Max. Drawdown Duration      456 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    52.33645
Equity Final [$]                  11979.26774
Equity Peak [$]                   13701.26407
Commissions [$]                      90.70783
Return [%]                           19.79268
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     6.26439
Volatility (Ann.) [%]                16.96957
CAGR [%]                              4.25956
Sharpe Ratio                          0.36915
Sortino Ratio                         0.57775
Calmar Ratio                          0.30236
Alpha [%]                             3.91445
Beta                                   0.3495
Max. Drawdown [%]                   -20.71803
Avg. Drawdown [%]                    -5.18858
Max. Drawdown Duration      316 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

...

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    22.02937
Equity Final [$]                  14256.70141
Equity Peak [$]                   15559.90057
Commissions [$]                      11.09727
Return [%]                           42.56701
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.67291
Volatility (Ann.) [%]                30.71886
CAGR [%]                              8.53641
Sharpe Ratio                          0.41254
Sortino Ratio                         0.68373
Calmar Ratio                          0.38244
Alpha [%]                            -2.21697
Beta                                  0.98575
Max. Drawdown [%]                   -33.13689
Avg. Drawdown [%]                    -7.05506
Max. Drawdown Duration      455 days 00:00:00
Avg. Drawdown Duration       61 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    51.26836
Equity Final [$]                   9347.73665
Equity Peak [$]                   12384.91933
Commissions [$]                      74.16909
Return [%]                           -6.52263
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    -2.24382
Volatility (Ann.) [%]                16.43306
CAGR [%]                             -1.54591
Sharpe Ratio                         -0.13654
Sortino Ratio                        -0.18233
Calmar Ratio                         -0.09097
Alpha [%]                           -22.78086
Beta                                  0.35787
Max. Drawdown [%]                    -24.6668
Avg. Drawdown [%]                    -6.77203
Max. Drawdown Duration      337 days 00:00:00
Avg. Drawdown Duration       88 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    55.40721
Equity Final [$]                   10507.9191
Equity Peak [$]                   12685.96262
Commissions [$]                      63.99572
Return [%]                            5.07919
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     1.68087
Volatility (Ann.) [%]                16.64723
CAGR [%]                              1.15095
Sharpe Ratio                          0.10097
Sortino Ratio                         0.14749
Calmar Ratio                          0.08436
Alpha [%]                           -11.36261
Beta                                  0.36191
Max. Drawdown [%]                   -19.92421
Avg. Drawdown [%]                    -5.67554
Max. Drawdown Duration      337 days 00:00:00
Avg. Drawdown Duration       81 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    61.68224
Equity Final [$]                  14097.94656
Equity Peak [$]                   15379.42573
Commissions [$]                      77.40136
Return [%]                           40.97947
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.24921
Volatility (Ann.) [%]                26.03006
CAGR [%]                              8.25604
Sharpe Ratio                          0.47058
Sortino Ratio                         0.77892
Calmar Ratio                          0.38187
Alpha [%]                             7.47389
Beta                                   0.7375
Max. Drawdown [%]                   -32.07703
Avg. Drawdown [%]                     -6.9128
Max. Drawdown Duration      664 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2022-03-04,2025-02-27,1091 days,55.006676,9894.180998,12249.298394,87.269280,-1.058190,45.431149,-0.357284,...,0.061844,145 days,26 days,1.111937,0.250118,0.050265,0.007721,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
1,2022-03-04,2025-02-27,1091 days,44.192256,12676.143434,12676.143434,105.006423,26.761434,45.431149,8.305343,...,0.958325,183 days,17 days,1.738316,1.335922,0.702292,0.205246,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
2,2022-03-04,2025-02-27,1091 days,27.636849,14093.888193,15375.367365,9.941782,40.938882,45.431149,12.238334,...,-7.266414,190 days,98 days,0.009044,-6.907929,-1.182126,-44.721951,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...
3,2022-03-04,2025-02-27,1091 days,52.336449,11979.267743,13701.264070,90.707834,19.792677,45.431149,6.264390,...,0.992927,144 days,26 days,1.565348,1.273079,0.613458,0.102120,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
4,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
5,2022-03-04,2025-02-27,1091 days,41.655541,14387.048765,14430.966071,101.174197,43.870488,45.431149,13.018454,...,1.642507,91 days,19 days,2.324098,1.848061,1.295195,0.291474,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
6,2022-03-04,2025-02-27,1091 days,64.886515,12404.808667,13154.358600,293.847148,24.048087,45.431149,7.519750,...,0.361976,128 days,9 days,1.485613,0.445068,0.771377,0.166333,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
7,2022-03-04,2025-02-27,1091 days,25.367156,10445.580834,11401.260217,59.401896,4.455808,45.431149,1.477520,...,-2.012973,80 days,13 days,0.207575,-1.945603,-2.163973,-1.543690,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
8,2022-03-04,2025-02-27,1091 days,22.029372,14256.701409,15559.900567,11.097268,42.567014,45.431149,12.672906,...,-2.304046,126 days,78 days,0.283372,-2.177661,-0.654040,-0.948784,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...
9,2022-03-04,2025-02-27,1091 days,51.268358,9347.736646,12384.919331,74.169088,-6.522634,45.431149,-2.243818,...,-0.168495,145 days,28 days,1.051990,0.149794,-0.086635,-0.021374,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...


In [11]:
import pandas as pd
pd.DataFrame(stats_l)

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2022-03-04,2025-02-27,1091 days,55.006676,9894.180998,12249.298394,87.269280,-1.058190,45.431149,-0.357284,...,0.061844,145 days,26 days,1.111937,0.250118,0.050265,0.007721,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
1,2022-03-04,2025-02-27,1091 days,44.192256,12676.143434,12676.143434,105.006423,26.761434,45.431149,8.305343,...,0.958325,183 days,17 days,1.738316,1.335922,0.702292,0.205246,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
2,2022-03-04,2025-02-27,1091 days,27.636849,14093.888193,15375.367365,9.941782,40.938882,45.431149,12.238334,...,-7.266414,190 days,98 days,0.009044,-6.907929,-1.182126,-44.721951,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...
3,2022-03-04,2025-02-27,1091 days,52.336449,11979.267743,13701.264070,90.707834,19.792677,45.431149,6.264390,...,0.992927,144 days,26 days,1.565348,1.273079,0.613458,0.102120,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
4,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
5,2022-03-04,2025-02-27,1091 days,41.655541,14387.048765,14430.966071,101.174197,43.870488,45.431149,13.018454,...,1.642507,91 days,19 days,2.324098,1.848061,1.295195,0.291474,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
6,2022-03-04,2025-02-27,1091 days,64.886515,12404.808667,13154.358600,293.847148,24.048087,45.431149,7.519750,...,0.361976,128 days,9 days,1.485613,0.445068,0.771377,0.166333,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
7,2022-03-04,2025-02-27,1091 days,25.367156,10445.580834,11401.260217,59.401896,4.455808,45.431149,1.477520,...,-2.012973,80 days,13 days,0.207575,-1.945603,-2.163973,-1.543690,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
8,2022-03-04,2025-02-27,1091 days,22.029372,14256.701409,15559.900567,11.097268,42.567014,45.431149,12.672906,...,-2.304046,126 days,78 days,0.283372,-2.177661,-0.654040,-0.948784,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...
9,2022-03-04,2025-02-27,1091 days,51.268358,9347.736646,12384.919331,74.169088,-6.522634,45.431149,-2.243818,...,-0.168495,145 days,28 days,1.051990,0.149794,-0.086635,-0.021374,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...


In [12]:
best_model = classifier[best_base]

In [13]:
y_pred = best_model.predict(X_test_scaled)

In [14]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749

0.8878504672897196
[[310  45]
 [ 39 355]]


In [15]:
!pip install backtesting

In [ ]:
X_train_scaled,X_test_scaled = scaling_function(X_train,X_test)
print(X_train_scaled.shape,X_test_scaled.shape,y_train.shape,y_test.shape)
classifier = classifier_models()
results,y_pred_dict = train_model(classifiers=classifier, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
best_base = results.loc[results['Accuracy'].idxmax(), 'Model']
param_grids = get_param_grids()
print("Working on optimizing with GridSearchCV")
best_models = grid_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
results,y_pred_dict = train_model(classifiers=best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)


(1745, 66) (749, 66) (1745,) (749,)

Logistic Regression Performance:
Accuracy: 0.8879
Confusion Matrix:
[[310  45]
 [ 39 355]]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       355
           1       0.89      0.90      0.89       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


Random Forest Performance:
Accuracy: 0.5955
Confusion Matrix:
[[284  71]
 [232 162]]
              precision    recall  f1-score   support

           0       0.55      0.80      0.65       355
           1       0.70      0.41      0.52       394

    accuracy                           0.60       749
   macro avg       0.62      0.61      0.58       749
weighted avg       0.63      0.60      0.58       749


XGBoost Performance:
Accuracy: 0.5394
Confusion Matrix:
[[ 10 345]
 [  0 394]]
              precision    recall  f1-score   suppor

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.6315
Confusion Matrix:
[[268  87]
 [189 205]]
              precision    recall  f1-score   support

           0       0.59      0.75      0.66       355
           1       0.70      0.52      0.60       394

    accuracy                           0.63       749
   macro avg       0.64      0.64      0.63       749
weighted avg       0.65      0.63      0.63       749


K-Nearest Neighbors Performance:
Accuracy: 0.6649
Confusion Matrix:
[[217 138]
 [113 281]]
              precision    recall  f1-score   support

           0       0.66      0.61      0.63       355
           1       0.67      0.71      0.69       394

    accuracy                           0.66       749
   macro avg       0.66      0.66      0.66       749
weighted avg       0.66      0.66      0.66       749


Decision Tree Performance:
Accuracy: 0.5794
Confusion Matrix:
[[ 59 296]
 [ 19 375]]
              precision    recall  f1-score   support

           0       0.76

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
param_grids = get_param_grids()
print("Working on optimizing with GridSearchCV")
best_models = grid_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
results,y_pred_dict = train_model(classifiers=best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)
grid_df = pd.DataFrame(stats_l)
param_grids = get_param_grids()
print("Working on optimizing with RandomSearchCV")
random_best_models = random_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
results,y_pred_dict = train_model(classifiers=random_best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)
random_df = pd.DataFrame(stats_l)




Working on optimizing with GridSearchCV
Optimizing Logistic Regression...


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for Logistic Regression: {'C': 10, 'penalty': 'l1', 'solver': 'saga'}
Optimizing Random Forest...
Best parameters for Random Forest: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 100}
Optimizing XGBoost...
Optimizing SVM...
Best parameters for SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Optimizing Naive Bayes...
Best parameters for Naive Bayes: {'alpha': 0.1}
Optimizing Gradient Boosting...
Best parameters for Gradient Boosting: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 10, 'n_estimators': 200}
Optimizing K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'algorithm': 'auto', 'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}
Optimizing Decision Tree...
Best parameters for Decision Tree: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 2}
Optimizing AdaBoost...
Best parameters for AdaBoost: {'learning_rate': 0.1, 'n_estimators': 50}
Optimizing SGD Classifier...

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\linear_model\_stochas

Best parameters for SGD Classifier: {'alpha': 0.001, 'learning_rate': 'optimal', 'loss': 'modified_huber', 'penalty': 'l1'}
Optimizing MLP...
Best parameters for MLP: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001, 'solver': 'adam'}
Optimizing LGBMClassifier...
[LightGBM] [Info] Number of positive: 1098, number of negative: 647
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001340 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16830
[LightGBM] [Info] Number of data points in the train set: 1745, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.629226 -> initscore=0.528899
[LightGBM] [Info] Start training from score 0.528899
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

...

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.6649
Confusion Matrix:
[[161 194]
 [ 57 337]]
              precision    recall  f1-score   support

           0       0.74      0.45      0.56       355
           1       0.63      0.86      0.73       394

    accuracy                           0.66       749
   macro avg       0.69      0.65      0.65       749
weighted avg       0.68      0.66      0.65       749


K-Nearest Neighbors Performance:
Accuracy: 0.6395
Confusion Matrix:
[[221 134]
 [136 258]]
              precision    recall  f1-score   support

           0       0.62      0.62      0.62       355
           1       0.66      0.65      0.66       394

    accuracy                           0.64       749
   macro avg       0.64      0.64      0.64       749
weighted avg       0.64      0.64      0.64       749


Decision Tree Performance:
Accuracy: 0.5260
Confusion Matrix:
[[  0 355]
 [  0 394]]
              precision    recall  f1-score   support

           0       0.00

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


AdaBoost Performance:
Accuracy: 0.5260
Confusion Matrix:
[[  0 355]
 [  0 394]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       355
           1       0.53      1.00      0.69       394

    accuracy                           0.53       749
   macro avg       0.26      0.50      0.34       749
weighted avg       0.28      0.53      0.36       749


SGD Classifier Performance:
Accuracy: 0.8905
Confusion Matrix:
[[317  38]
 [ 44 350]]
              precision    recall  f1-score   support

           0       0.88      0.89      0.89       355
           1       0.90      0.89      0.90       394

    accuracy                           0.89       749
   macro avg       0.89      0.89      0.89       749
weighted avg       0.89      0.89      0.89       749


MLP Performance:
Accuracy: 0.8411
Confusion Matrix:
[[270  85]
 [ 34 360]]
              precision    recall  f1-score   support

           0       0.89      0.76      0.82    

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)
grid_df = pd.DataFrame(stats_l)
param_grids = get_param_grids()
print("Working on optimizing with RandomSearchCV")
random_best_models = random_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
results,y_pred_dict = train_model(classifiers=random_best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)




Backtesting for Logistic Regression


Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    54.87316
Equity Final [$]                   9565.93438
Equity Peak [$]                   12249.29839
Commissions [$]                      94.91414
Return [%]                           -4.34066
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    -1.48196
Volatility (Ann.) [%]                17.06199
CAGR [%]                             -1.01978
Sharpe Ratio                         -0.08686
Sortino Ratio                        -0.11763
Calmar Ratio                         -0.06158
Alpha [%]                           -21.92115
Beta                                  0.38697
Max. Drawdown [%]                    -24.0661
Avg. Drawdown [%]                    -6.94142
Max. Drawdown Duration      344 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                  14863.79037
Equity Peak [$]                    16210.4295
Return [%]                            48.6379
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    14.26488
Volatility (Ann.) [%]                 30.9169
CAGR [%]                              9.58689
Sharpe Ratio                          0.46139
Sortino Ratio                         0.77786
Calmar Ratio                          0.47972
Alpha [%]                             3.99676
Beta                                  0.98261
Max. Drawdown [%]                   -29.73563
Avg. Drawdown [%]                    -5.70212
Max. Drawdown Duration      429 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                    0
Win Rate [%]                      

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    27.63685
Equity Final [$]                  14093.88819
Equity Peak [$]                   15375.36737
Commissions [$]                       9.94178
Return [%]                           40.93888
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.23833
Volatility (Ann.) [%]                30.08641
CAGR [%]                              8.24884
Sharpe Ratio                          0.40677
Sortino Ratio                         0.67242
Calmar Ratio                           0.4172
Alpha [%]                            -2.71274
Beta                                  0.96083
Max. Drawdown [%]                   -29.33467
Avg. Drawdown [%]                    -6.80479
Max. Drawdown Duration      456 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    52.33645
Equity Final [$]                  11979.26774
Equity Peak [$]                   13701.26407
Commissions [$]                      90.70783
Return [%]                           19.79268
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     6.26439
Volatility (Ann.) [%]                16.96957
CAGR [%]                              4.25956
Sharpe Ratio                          0.36915
Sortino Ratio                         0.57775
Calmar Ratio                          0.30236
Alpha [%]                             3.91445
Beta                                   0.3495
Max. Drawdown [%]                   -20.71803
Avg. Drawdown [%]                    -5.18858
Max. Drawdown Duration      316 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

...

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                  14863.79037
Equity Peak [$]                    16210.4295
Return [%]                            48.6379
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    14.26488
Volatility (Ann.) [%]                 30.9169
CAGR [%]                              9.58689
Sharpe Ratio                          0.46139
Sortino Ratio                         0.77786
Calmar Ratio                          0.47972
Alpha [%]                             3.99676
Beta                                  0.98261
Max. Drawdown [%]                   -29.73563
Avg. Drawdown [%]                    -5.70212
Max. Drawdown Duration      429 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                    0
Win Rate [%]                      

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                  14863.79037
Equity Peak [$]                    16210.4295
Return [%]                            48.6379
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    14.26488
Volatility (Ann.) [%]                 30.9169
CAGR [%]                              9.58689
Sharpe Ratio                          0.46139
Sortino Ratio                         0.77786
Calmar Ratio                          0.47972
Alpha [%]                             3.99676
Beta                                  0.98261
Max. Drawdown [%]                   -29.73563
Avg. Drawdown [%]                    -5.70212
Max. Drawdown Duration      429 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                    0
Win Rate [%]                      

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    53.27103
Equity Final [$]                  10368.37969
Equity Peak [$]                   11997.22932
Commissions [$]                       83.7376
Return [%]                             3.6838
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     1.22456
Volatility (Ann.) [%]                16.47692
CAGR [%]                              0.83909
Sharpe Ratio                          0.07432
Sortino Ratio                         0.10853
Calmar Ratio                          0.05465
Alpha [%]                           -12.68863
Beta                                  0.36038
Max. Drawdown [%]                   -22.40812
Avg. Drawdown [%]                    -6.39092
Max. Drawdown Duration      691 days 00:00:00
Avg. Drawdown Duration      133 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    59.41255
Equity Final [$]                  11811.48357
Equity Peak [$]                   13874.00549
Commissions [$]                      62.79558
Return [%]                           18.11484
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     5.76129
Volatility (Ann.) [%]                18.35209
CAGR [%]                              3.92043
Sharpe Ratio                          0.31393
Sortino Ratio                         0.47866
Calmar Ratio                          0.28928
Alpha [%]                             0.07019
Beta                                  0.39719
Max. Drawdown [%]                   -19.91623
Avg. Drawdown [%]                    -5.09136
Max. Drawdown Duration      318 days 00:00:00
Avg. Drawdown Duration       55 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    61.68224
Equity Final [$]                  14097.94656
Equity Peak [$]                   15379.42573
Commissions [$]                      77.40136
Return [%]                           40.97947
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.24921
Volatility (Ann.) [%]                26.03006
CAGR [%]                              8.25604
Sharpe Ratio                          0.47058
Sortino Ratio                         0.77892
Calmar Ratio                          0.38187
Alpha [%]                             7.47389
Beta                                   0.7375
Max. Drawdown [%]                   -32.07703
Avg. Drawdown [%]                     -6.9128
Max. Drawdown Duration      664 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

In [19]:

import pandas as pd
pd.DataFrame(stats_l)

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2022-03-04,2025-02-27,1091 days,54.873164,9565.934383,12249.298394,94.914136,-4.340656,45.431149,-1.481961,...,-0.084792,145 days,24 days,1.068750,0.176051,-0.049592,-0.007275,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
1,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
2,2022-03-04,2025-02-27,1091 days,27.636849,14093.888193,15375.367365,9.941782,40.938882,45.431149,12.238334,...,-7.266414,190 days,98 days,0.009044,-6.907929,-1.182126,-44.721951,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...
3,2022-03-04,2025-02-27,1091 days,52.336449,11979.267743,13701.264070,90.707834,19.792677,45.431149,6.264390,...,0.992927,144 days,26 days,1.565348,1.273079,0.613458,0.102120,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
4,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
5,2022-03-04,2025-02-27,1091 days,51.535381,13173.312682,14367.911910,105.897852,31.733127,45.431149,9.716316,...,0.623314,160 days,21 days,1.513888,0.805080,0.557829,0.157192,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
6,2022-03-04,2025-02-27,1091 days,59.946595,9162.904819,11450.509530,210.584714,-8.370952,45.431149,-2.898461,...,-0.120530,94 days,11 days,0.971335,-0.036693,-0.221261,-0.045724,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
7,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
8,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
9,2022-03-04,2025-02-27,1091 days,53.271028,10368.379693,11997.229324,83.737604,3.683797,45.431149,1.224562,...,0.290865,143 days,27 days,1.217347,0.526905,0.197661,0.035928,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...


In [ ]:
param_grids = get_param_grids()
print("Working on optimizing with RandomSearchCV")
random_best_models = random_optimize_model(classifier,X_train_scaled, y_train,param_grids=param_grids)
results,y_pred_dict = train_model(classifiers=random_best_models, X_train_scaled=X_train_scaled, X_test_scaled=X_test_scaled,y_train=y_train,y_test=y_test)
print(results)
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)



Working on optimizing with RandomSearchCV
Optimizing Logistic Regression...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'C': 10}
Optimizing Random Forest...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Random Forest: {'n_estimators': 50, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 10}
Optimizing XGBoost...
No hyperparameters to tune for XGBoost
Optimizing SVM...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for SVM: {'kernel': 'linear', 'gamma': 'auto', 'C': 10}
Optimizing Naive Bayes...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters for Naive Bayes: {'alpha': 0.1}
Optimizing Gradient Boosting...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for Gradient Boosting: {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': 5, 'learning_rate': 0.05}
Optimizing K-Nearest Neighbors...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for K-Nearest Neighbors: {'weights': 'distance', 'p': 2, 'n_neighbors': 7, 'algorithm': 'kd_tree'}
Optimizing Decision Tree...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Decision Tree: {'min_samples_split': 10, 'max_features': None, 'max_depth': 5, 'criterion': 'entropy'}
Optimizing AdaBoost...
Fitting 5 folds for each of 6 candidates, totalling 30 fits


...

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


Gradient Boosting Performance:
Accuracy: 0.6716
Confusion Matrix:
[[168 187]
 [ 59 335]]
              precision    recall  f1-score   support

           0       0.74      0.47      0.58       355
           1       0.64      0.85      0.73       394

    accuracy                           0.67       749
   macro avg       0.69      0.66      0.65       749
weighted avg       0.69      0.67      0.66       749


K-Nearest Neighbors Performance:
Accuracy: 0.6395
Confusion Matrix:
[[221 134]
 [136 258]]
              precision    recall  f1-score   support

           0       0.62      0.62      0.62       355
           1       0.66      0.65      0.66       394

    accuracy                           0.64       749
   macro avg       0.64      0.64      0.64       749
weighted avg       0.64      0.64      0.64       749


Decision Tree Performance:
Accuracy: 0.6088
Confusion Matrix:
[[106 249]
 [ 44 350]]
              precision    recall  f1-score   support

           0       0.71

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita


AdaBoost Performance:
Accuracy: 0.5260
Confusion Matrix:
[[  0 355]
 [  0 394]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       355
           1       0.53      1.00      0.69       394

    accuracy                           0.53       749
   macro avg       0.26      0.50      0.34       749
weighted avg       0.28      0.53      0.36       749


SGD Classifier Performance:
Accuracy: 0.8785
Confusion Matrix:
[[338  17]
 [ 74 320]]
              precision    recall  f1-score   support

           0       0.82      0.95      0.88       355
           1       0.95      0.81      0.88       394

    accuracy                           0.88       749
   macro avg       0.88      0.88      0.88       749
weighted avg       0.89      0.88      0.88       749


MLP Performance:
Accuracy: 0.8251
Confusion Matrix:
[[246 109]
 [ 22 372]]
              precision    recall  f1-score   support

           0       0.92      0.69      0.79    

c:\Users\sachi\Documents\Researchcode\research\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [24]:
stats_l=[]
for name,y_pred in y_pred_dict.items():
    print(f"Backtesting for {name}")
    stats = backtest(y_pred,X_test)
    print(stats)
    stats_l.append(stats)



Backtesting for Logistic Regression


Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    54.87316
Equity Final [$]                   9565.93438
Equity Peak [$]                   12249.29839
Commissions [$]                      94.91414
Return [%]                           -4.34066
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    -1.48196
Volatility (Ann.) [%]                17.06199
CAGR [%]                             -1.01978
Sharpe Ratio                         -0.08686
Sortino Ratio                        -0.11763
Calmar Ratio                         -0.06158
Alpha [%]                           -21.92115
Beta                                  0.38697
Max. Drawdown [%]                    -24.0661
Avg. Drawdown [%]                    -6.94142
Max. Drawdown Duration      344 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                      35.247
Equity Final [$]                  11210.06569
Equity Peak [$]                   11288.60685
Commissions [$]                      68.61411
Return [%]                           12.10066
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     3.91796
Volatility (Ann.) [%]                15.46173
CAGR [%]                              2.67354
Sharpe Ratio                           0.2534
Sortino Ratio                         0.37743
Calmar Ratio                          0.15721
Alpha [%]                            -2.00516
Beta                                  0.31049
Max. Drawdown [%]                   -24.92139
Avg. Drawdown [%]                    -4.08238
Max. Drawdown Duration      421 days 00:00:00
Avg. Drawdown Duration      132 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    27.63685
Equity Final [$]                  14093.88819
Equity Peak [$]                   15375.36737
Commissions [$]                       9.94178
Return [%]                           40.93888
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.23833
Volatility (Ann.) [%]                30.08641
CAGR [%]                              8.24884
Sharpe Ratio                          0.40677
Sortino Ratio                         0.67242
Calmar Ratio                           0.4172
Alpha [%]                            -2.71274
Beta                                  0.96083
Max. Drawdown [%]                   -29.33467
Avg. Drawdown [%]                    -6.80479
Max. Drawdown Duration      456 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    52.33645
Equity Final [$]                  11979.26774
Equity Peak [$]                   13701.26407
Commissions [$]                      90.70783
Return [%]                           19.79268
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     6.26439
Volatility (Ann.) [%]                16.96957
CAGR [%]                              4.25956
Sharpe Ratio                          0.36915
Sortino Ratio                         0.57775
Calmar Ratio                          0.30236
Alpha [%]                             3.91445
Beta                                   0.3495
Max. Drawdown [%]                   -20.71803
Avg. Drawdown [%]                    -5.18858
Max. Drawdown Duration      316 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

...

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                     55.2737
Equity Final [$]                  16149.27427
Equity Peak [$]                   17626.23332
Commissions [$]                     111.21991
Return [%]                           61.49274
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    17.49863
Volatility (Ann.) [%]                25.63827
CAGR [%]                             11.70673
Sharpe Ratio                          0.68252
Sortino Ratio                         1.21936
Calmar Ratio                          1.01723
Alpha [%]                            32.05584
Beta                                  0.64795
Max. Drawdown [%]                    -17.2022
Avg. Drawdown [%]                    -3.68705
Max. Drawdown Duration      316 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                  14863.79037
Equity Peak [$]                    16210.4295
Return [%]                            48.6379
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    14.26488
Volatility (Ann.) [%]                 30.9169
CAGR [%]                              9.58689
Sharpe Ratio                          0.46139
Sortino Ratio                         0.77786
Calmar Ratio                          0.47972
Alpha [%]                             3.99676
Beta                                  0.98261
Max. Drawdown [%]                   -29.73563
Avg. Drawdown [%]                    -5.70212
Max. Drawdown Duration      429 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                    0
Win Rate [%]                      

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    46.06142
Equity Final [$]                  12791.73742
Equity Peak [$]                   13886.54408
Commissions [$]                      76.64545
Return [%]                           27.91737
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     8.63663
Volatility (Ann.) [%]                16.03023
CAGR [%]                               5.8519
Sharpe Ratio                          0.53877
Sortino Ratio                         0.88827
Calmar Ratio                           0.5301
Alpha [%]                            14.44341
Beta                                  0.29658
Max. Drawdown [%]                   -16.29243
Avg. Drawdown [%]                    -4.52897
Max. Drawdown Duration      316 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    63.41789
Equity Final [$]                  11081.28371
Equity Peak [$]                    12858.8723
Commissions [$]                      55.46154
Return [%]                           10.81284
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                     3.51476
Volatility (Ann.) [%]                18.78102
CAGR [%]                              2.39988
Sharpe Ratio                          0.18714
Sortino Ratio                         0.27524
Calmar Ratio                          0.15024
Alpha [%]                            -8.60619
Beta                                  0.42744
Max. Drawdown [%]                   -23.39486
Avg. Drawdown [%]                     -5.1006
Max. Drawdown Duration      497 days 00:00:00
Avg. Drawdown Duration       66 days 00:00:00
# Trades                          

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    61.68224
Equity Final [$]                  14097.94656
Equity Peak [$]                   15379.42573
Commissions [$]                      77.40136
Return [%]                           40.97947
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    12.24921
Volatility (Ann.) [%]                26.03006
CAGR [%]                              8.25604
Sharpe Ratio                          0.47058
Sortino Ratio                         0.77892
Calmar Ratio                          0.38187
Alpha [%]                             7.47389
Beta                                   0.7375
Max. Drawdown [%]                   -32.07703
Avg. Drawdown [%]                     -6.9128
Max. Drawdown Duration      664 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                          

In [25]:

import pandas as pd
pd.DataFrame(stats_l)

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Commissions [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2022-03-04,2025-02-27,1091 days,54.873164,9565.934383,12249.298394,94.914136,-4.340656,45.431149,-1.481961,...,-0.084792,145 days,24 days,1.068750,0.176051,-0.049592,-0.007275,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
1,2022-03-04,2025-02-27,1091 days,35.246996,11210.065686,11288.606855,68.614108,12.100657,45.431149,3.917956,...,0.688389,186 days,20 days,1.666826,1.145205,0.354108,0.139648,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
2,2022-03-04,2025-02-27,1091 days,27.636849,14093.888193,15375.367365,9.941782,40.938882,45.431149,12.238334,...,-7.266414,190 days,98 days,0.009044,-6.907929,-1.182126,-44.721951,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice ExitP...
3,2022-03-04,2025-02-27,1091 days,52.336449,11979.267743,13701.264070,90.707834,19.792677,45.431149,6.264390,...,0.992927,144 days,26 days,1.565348,1.273079,0.613458,0.102120,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
4,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
5,2022-03-04,2025-02-27,1091 days,57.810414,11990.305282,13076.304580,69.568924,19.903053,45.431149,6.297322,...,0.818769,225 days,32 days,1.593378,1.161492,0.471945,0.163034,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
6,2022-03-04,2025-02-27,1091 days,59.946595,9162.904819,11450.509530,210.584714,-8.370952,45.431149,-2.898461,...,-0.120530,94 days,11 days,0.971335,-0.036693,-0.221261,-0.045724,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
7,2022-03-04,2025-02-27,1091 days,55.273698,16149.274274,17626.233319,111.219907,61.492743,45.431149,17.498626,...,0.577265,175 days,21 days,1.670142,0.657030,0.756620,0.152491,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...
8,2022-03-04,2025-02-27,1091 days,0.000000,14863.790367,16210.429497,NaN,48.637904,45.431149,14.264876,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,MyStrategy,Equity DrawdownPct Drawdown...,"Empty DataFrame Columns: [Size, EntryBar, Exit..."
9,2022-03-04,2025-02-27,1091 days,46.061415,12791.737424,13886.544081,76.645450,27.917374,45.431149,8.636632,...,1.517690,98 days,27 days,2.012773,1.727632,1.011004,0.243059,MyStrategy,Equity DrawdownPct Drawdown...,Size EntryBar ExitBar EntryPrice Exit...


In [26]:
from backtesting import Backtest,Strategy
def get_signal():
    return y_test
class MyStrategy(Strategy):
    def init(self):
        self.signal = self.I(get_signal)
    def next(self):
        if self.signal == 1:
            if not self.position:
                self.buy()
        elif self.signal == 0:
            if self.position:
                self.position.close()

bt = Backtest(X_test,MyStrategy,cash=10000,commission=0.0002)
stats = bt.run()
stats

Start                     2022-03-04 00:00:00
End                       2025-02-27 00:00:00
Duration                   1091 days 00:00:00
Exposure Time [%]                    51.26836
Equity Final [$]                  17462.47122
Equity Peak [$]                   18473.92066
Commissions [$]                      69.04271
Return [%]                           74.62471
Buy & Hold Return [%]                45.43115
Return (Ann.) [%]                    20.63022
Volatility (Ann.) [%]                20.40953
CAGR [%]                             13.74223
Sharpe Ratio                          1.01081
Sortino Ratio                         1.94515
Calmar Ratio                          1.54451
Alpha [%]                             57.2394
Beta                                  0.38267
Max. Drawdown [%]                   -13.35712
Avg. Drawdown [%]                    -2.91981
Max. Drawdown Duration      226 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                          